In [1]:
from sklearn.ensemble import RandomForestClassifier
from src.hpo_module import HPO_runner
# === HPO Runner ===


if __name__ == "__main__":
    best_params, best_accuracy, accuracy = HPO_runner(
        file_path="data/FraudDetectionDataset.xlsx",
        target_column='Fraudulent',
        model=RandomForestClassifier(),  # Model Object
        params={
            'n_estimators': [5, 50],  # Hyperparameters with ranges
        },
        task_type="classification", 
        SearchAgents=5,
        Max_iterations=5,
        test_size=0.5
    )
print(accuracy)

Best Parameters for RandomForestClassifier: {'n_estimators': np.float64(46.09279485612239)}
Best Accuracy: 0.5279


NameError: name 'best_fit' is not defined